In [1]:
from words import *
from top_dense import *
from wordbank import *
import json
import os

In [2]:
%load_ext autoreload
%autoreload 2

## Load the word vectors

We're using the 42 billion token, 300D, GloVe model, 
(glove.42B.300d.zip from http://nlp.stanford.edu/projects/glove/)
with 1.9 million words in the vocab. The original 
vector file was parsed into a numpy array and that 
numpy array was saved to disc. The function that does
this parsing/loading is words.create_numpy_from_glove()
We load that numpy array and the vocab dictionary from
the output of that function here.

In [8]:
glove = GloVe(vocab="data/model/dict_glove_42b_300", 
              vectors="data/model/vectors_glove_42b_300.npy")

### Load the WordBank data

Load the Wordbank data as a measure of infants' vocabulary. We'll pass these words into the semantic graph generator. We'll do the same thing to the environmental linguistic input.

In [9]:
# load all the WordBank words for english
wordbank_english = WordBank(input="data/wb_cdi/wordbank_english.csv")


# wordmap is a dictionary of keys to words.
# here key is just the name of the input dataset, 
# i.e. wordbank_english
wb_eng_wordmap = wordbank_english.wordmap()


### Generate a semantic graph 

For every word, give it an edge with any other word that's within some threshold of cos(θ) to it (i.e.  cos(θ) >= 1.0 - threshold ). Output these graphs in JSON form with each word as a key, and a list of its neighbors as the value. We're generating a distinct graph for a range of threshold values. The "start" and "end" parameters specify this range and the "step" parameter specifies the granularity. 

In [13]:
glove.graph_cosine_range(output_path="english_wordbank", wordmap=wb_eng_wordmap, 
                         start=0.3, end=0.33, step=0.01)

### Rank nodes in each graph by their degree

For each of the graphs that we generated in the previous step, we output a csv file of the top ranked words (by degree). 

In [4]:
rank_density(input_path="data/output/english_wordbank/semgraphs", 
             output_path="data/output/english_wordbank/ranked_out")